In [36]:
import pandas as pd
import math
import numpy as np
import requests
#!conda install -c conda-forge -y reverse-geocode 
import reverse_geocode
import pyproj
from env_tools import prepare_env #custom local site library that is used to prepare environment variables with 
                                  #sensitive information without showing such information explicitly
prepare_env('google')
import os
key = os.environ['GEOCODERS_API'] # google offers a geocoders api available for use for a fee
#!pip install reverse-geocode
counties_df = pd.read_csv("/home/sbr0/Dropbox/Analysis Planning/coursera stuffs/Capstone/First-Class Restaurant Location Analysis/data sets/Counties Dataset.csv")

In [16]:
counties_df

,State,Raw County Data (2015),Violent crime (2015),Murder and nonnegligent manslaughter (2015),Rape (revised) (2015),Rape (previous def) (2015),Robbery (2015),Aggravated assault (2015),Property crime (2015),Burglary (2015),...,Unemployed_2014,Employed_2015,Unemployed_2015,Employed_2016,Unemployed_2016,Employed_2017,Unemployed_2017,Geographical Area (sq mi),Housing Density (per sq mi),Population
0,Alabama,Autauga,69.0,0.0,13.0,NaN,6.0,50.0,344.0,111.0,...,1495.0,24272.0,1330.0,24593.0,1325.0,24908.0,1001.0,604.39,37.2,54571
1,Alabama,Baldwin,115.0,0.0,9.0,NaN,33.0,73.0,648.0,225.0,...,5300.0,82843.0,4862.0,85656.0,4844.0,87915.0,3652.0,2027.31,65.5,182265
2,Alabama,Bibb,7.0,0.0,2.0,NaN,1.0,4.0,41.0,20.0,...,617.0,8005.0,567.0,8050.0,557.0,8133.0,373.0,626.17,14.4,22915
3,Alabama,Blount,204.0,5.0,16.0,NaN,5.0,178.0,832.0,247.0,...,1503.0,23152.0,1321.0,23248.0,1328.0,23509.0,985.0,650.63,37.0,57322
4,Alabama,Calhoun,16.0,0.0,4.0,NaN,1.0,11.0,413.0,181.0,...,3773.0,42679.0,3231.0,42815.0,3003.0,43333.0,2249.0,612.29,88.0,118572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2767,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,164.0,2596.0,159.0,2565.0,136.0,2555.0,125.0,675.29,7.6,6306
2768,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,609.0,15338.0,545.0,15396.0,495.0,15445.0,489.0,692.27,21.6,28936
2769,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1324.0,32849.0,1188.0,32804.0,1083.0,33115.0,1009.0,695.42,43.6,59534
2770,Vermont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1093.0,28501.0,948.0,28454.0,862.0,28508.0,750.0,976.78,35.2,56670


In [37]:
try:
    coordinate_dataframe = pd.read_csv("County Coordinates.csv")
    
except FileNotFoundError:
    
    index=['Area',"Google County Name","Latitude","Longitude","Northeast Bound","Southwest Bound"]
    
    # Using the google maps api directly to reliably retrieve location data by postal code:
    def get_coords(area_name,index=index,key=key):
        params = dict(key=key,address="{}, United States".format(area_name))
        api_url = 'https://maps.googleapis.com/maps/api/geocode/json'
        response = requests.get(api_url,params=params)    
        try:
            location=response.json()['results'][0]['geometry']['location']
            lat, lon = location['lat'],location['lng']
            google_county_name=response.json()['results'][0]['address_components'][0]['long_name']

            ne_lat_lon_result=response.json()['results'][0]['geometry']['bounds']['northeast']
            ne_lat_lon_tuple = (ne_lat_lon_result['lat'],ne_lat_lon_result['lng'])

            sw_lat_lon_result=response.json()['results'][0]['geometry']['bounds']['southwest']
            sw_lat_lon_tuple = (sw_lat_lon_result['lat'],sw_lat_lon_result['lng'])

            row_vals = [area_name,google_county_name,lat,lon,ne_lat_lon_tuple,sw_lat_lon_tuple]
            row = pd.Series(row_vals,index=index)
        except:
            print("Error with County",area_name+'\n', 'data did not process')
            row_vals = [area_name] + [np.nan] * 5
            row = pd.Series(row_vals,index=index)
        return row

    coordinate_dataframe=counties_df['Area'].apply(get_coords)
    
    # to avoid paying money unnecessarily, the dataset is saved for later use when running the same analysis:
    coordinate_dataframe.to_csv("County Coordinates.csv",index=False)

print("Data Types:")
print(coordinate_dataframe.dtypes,'\n')
print("Total Null:")
print(coordinate_dataframe.isnull().sum(),'\n\n')
print("Coordinate DataFrame:")
display(coordinate_dataframe)

Data Types:
Area                   object
Google County Name     object
Latitude              float64
Longitude             float64
Northeast Bound        object
Southwest Bound        object
dtype: object 

Total Null:
Area                  0
Google County Name    0
Latitude              0
Longitude             0
Northeast Bound       0
Southwest Bound       0
dtype: int64 


Coordinate DataFrame:


,Area,Google County Name,Latitude,Longitude,Northeast Bound,Southwest Bound
0,"Autauga, Alabama",Autauga County,32.579182,-86.499655,"(32.7082131, -86.411172)","(32.307574, -86.9212369)"
1,"Baldwin, Alabama",Baldwin County,30.601074,-87.776333,"(31.3190171, -87.366591)","(30.1465611, -88.0373089)"
2,"Bibb, Alabama",Bibb County,32.956280,-87.142289,"(33.246926, -86.8760228)","(32.83152210000001, -87.421989)"
3,"Blount, Alabama",Blount County,34.014515,-86.499655,"(34.2604829, -86.3034978)","(33.765171, -86.963358)"
4,"Calhoun, Alabama",Calhoun County,33.770158,-85.807660,"(33.96846, -85.52943210000001)","(33.5559299, -86.146225)"
...,...,...,...,...,...,...
2863,"Gilmer, West Virginia",Gilmer County,38.920331,-80.854385,"(39.1094009, -80.60579899999999)","(38.717898, -81.05671389999999)"
2864,"Pleasants, West Virginia",Pleasants County,39.376171,-81.141669,"(39.4764319, -81.00783589999999)","(39.2683278, -81.371478)"
2865,"Pocahontas, West Virginia",Pocahontas County,38.310171,-80.008775,"(38.739811, -79.61787509999999)","(38.03611, -80.3632959)"
2866,"Ritchie, West Virginia",Ritchie County,39.182230,-81.031299,"(39.38911909999999, -80.80888399999999)","(39.007884, -81.329228)"


In [27]:
def get_remaining_coords(area_data,index=index,key=key):
    if area_data.isnull().any():
        county, state = area_data['Area'].rsplit(", ",1)
        area_name = county + " County, " + state
        params = dict(key=key,address="{}, United States".format(area_name))
        api_url = 'https://maps.googleapis.com/maps/api/geocode/json'
        response = requests.get(api_url,params=params)    
        try:
            location=response.json()['results'][0]['geometry']['location']
            lat, lon = location['lat'],location['lng']
            google_county_name=response.json()['results'][0]['address_components'][0]['long_name']

            ne_lat_lon_result=response.json()['results'][0]['geometry']['bounds']['northeast']
            ne_lat_lon_tuple = (ne_lat_lon_result['lat'],ne_lat_lon_result['lng'])

            sw_lat_lon_result=response.json()['results'][0]['geometry']['bounds']['southwest']
            sw_lat_lon_tuple = (sw_lat_lon_result['lat'],sw_lat_lon_result['lng'])

            row_vals = [area_name,google_county_name,lat,lon,ne_lat_lon_tuple,sw_lat_lon_tuple]
            area_data = pd.Series(row_vals,index=index)
        except:
            print("Error with County",area_name+'\n', 'data did not process')
            row_vals = [area_name] + [np.nan] * 5
            area_data = pd.Series(row_vals,index=index)
    return area_data

coordinate_dataframe=coordinate_dataframe.apply(get_remaining_coords,axis=1)
    
    # to avoid paying money unnecessarily, the dataset is saved for later use when running the same analysis:
    #coordinate_dataframe.to_csv("County Coordinates.csv",index=False)

In [35]:
# to avoid paying money unnecessarily, the dataset is saved for later use when running the same analysis:
#coordinate_dataframe.to_csv("County Coordinates.csv",index=False)
coordinate_dataframe.to_csv("County Coordinates.csv",index=False)

In [39]:
assert pd.merge(counties_df,coordinate_dataframe).duplicated().sum() == 0

In [40]:
display(coordinate_dataframe.isnull().sum())
coordinate_dataframe

Area                  0
Google County Name    0
Latitude              0
Longitude             0
Northeast Bound       0
Southwest Bound       0
dtype: int64

,Area,Google County Name,Latitude,Longitude,Northeast Bound,Southwest Bound
0,"Autauga, Alabama",Autauga County,32.579182,-86.499655,"(32.7082131, -86.411172)","(32.307574, -86.9212369)"
1,"Baldwin, Alabama",Baldwin County,30.601074,-87.776333,"(31.3190171, -87.366591)","(30.1465611, -88.0373089)"
2,"Bibb, Alabama",Bibb County,32.956280,-87.142289,"(33.246926, -86.8760228)","(32.83152210000001, -87.421989)"
3,"Blount, Alabama",Blount County,34.014515,-86.499655,"(34.2604829, -86.3034978)","(33.765171, -86.963358)"
4,"Calhoun, Alabama",Calhoun County,33.770158,-85.807660,"(33.96846, -85.52943210000001)","(33.5559299, -86.146225)"
...,...,...,...,...,...,...
2863,"Gilmer, West Virginia",Gilmer County,38.920331,-80.854385,"(39.1094009, -80.60579899999999)","(38.717898, -81.05671389999999)"
2864,"Pleasants, West Virginia",Pleasants County,39.376171,-81.141669,"(39.4764319, -81.00783589999999)","(39.2683278, -81.371478)"
2865,"Pocahontas, West Virginia",Pocahontas County,38.310171,-80.008775,"(38.739811, -79.61787509999999)","(38.03611, -80.3632959)"
2866,"Ritchie, West Virginia",Ritchie County,39.182230,-81.031299,"(39.38911909999999, -80.80888399999999)","(39.007884, -81.329228)"


In [41]:
coordinate_dataframe = pd.read_csv("County Coordinates.csv")

In [42]:
coordinate_dataframe

,Area,Google County Name,Latitude,Longitude,Northeast Bound,Southwest Bound
0,"Autauga, Alabama",Autauga County,32.579182,-86.499655,"(32.7082131, -86.411172)","(32.307574, -86.9212369)"
1,"Baldwin, Alabama",Baldwin County,30.601074,-87.776333,"(31.3190171, -87.366591)","(30.1465611, -88.0373089)"
2,"Bibb, Alabama",Bibb County,32.956280,-87.142289,"(33.246926, -86.8760228)","(32.83152210000001, -87.421989)"
3,"Blount, Alabama",Blount County,34.014515,-86.499655,"(34.2604829, -86.3034978)","(33.765171, -86.963358)"
4,"Calhoun, Alabama",Calhoun County,33.770158,-85.807660,"(33.96846, -85.52943210000001)","(33.5559299, -86.146225)"
...,...,...,...,...,...,...
2863,"Gilmer, West Virginia",Gilmer County,38.920331,-80.854385,"(39.1094009, -80.60579899999999)","(38.717898, -81.05671389999999)"
2864,"Pleasants, West Virginia",Pleasants County,39.376171,-81.141669,"(39.4764319, -81.00783589999999)","(39.2683278, -81.371478)"
2865,"Pocahontas, West Virginia",Pocahontas County,38.310171,-80.008775,"(38.739811, -79.61787509999999)","(38.03611, -80.3632959)"
2866,"Ritchie, West Virginia",Ritchie County,39.182230,-81.031299,"(39.38911909999999, -80.80888399999999)","(39.007884, -81.329228)"


In [43]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

# results in projection distortion:
# def calc_xy_distance(x1, y1, x2, y2):
#     dx = x2 - x1
#     dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)
def parse_string_latlon(latlon, order_to_lonlat=False):
    latlon_str = latlon.replace("(",'').replace(")",'').replace(" ",'')
    lat, lon = latlon_str.split(",")
    
    coords = (float(lat), float(lon)) if not order_to_lonlat else (float(lon), float(lat))
    return coords
    
    #tuple(x for (x,y) in latlon.replace("(",'').split(","))

In [44]:
coordinate_dataframe["Northeast Bound lonlat"]=coordinate_dataframe["Northeast Bound"].apply(lambda strlatlon: parse_string_latlon(strlatlon,order_to_lonlat=True))
coordinate_dataframe["Southwest Bound lonlat"]=coordinate_dataframe["Southwest Bound"].apply(lambda strlatlon: parse_string_latlon(strlatlon,order_to_lonlat=True))
coordinate_dataframe["Northeast Bound xy"]=coordinate_dataframe["Northeast Bound lonlat"].apply(lambda lonlat: lonlat_to_xy(lonlat[0],lonlat[1]))
coordinate_dataframe["Southwest Bound xy"]=coordinate_dataframe["Southwest Bound lonlat"].apply(lambda lonlat: lonlat_to_xy(lonlat[0],lonlat[1]))


#coordinate_dataframe["Northeast Bound"].apply(lambda x:tuple(x))

In [46]:
def get_county_distances(coord_obs):
    x1 = coord_obs["Southwest Bound xy"][0]
    y1 = coord_obs["Southwest Bound xy"][1]
    x2 = coord_obs["Northeast Bound xy"][0]
    y2 = coord_obs["Northeast Bound xy"][1]
    return calc_xy_distance(x1,y1,x2,y2)

G = pyproj.Geod(ellps='WGS84')

def get_county_geodetic_distances(coord_obs,):
    lon1 = coord_obs["Southwest Bound lonlat"][0]
    lat1 = coord_obs["Southwest Bound lonlat"][1]
    lon2 = coord_obs["Northeast Bound lonlat"][0]
    lat2 = coord_obs["Northeast Bound lonlat"][1]
    return G.inv(lon1, lat1, lon2, lat2)[2]

def get_county_geodetic_distances_ll_centered(coord_obs,):
    lon1 = coord_obs["Southwest Bound lonlat"][0]
    lat1 = coord_obs["Southwest Bound lonlat"][1]
    lon2 = coord_obs["Longitude"]
    lat2 = coord_obs["Latitude"]
    #lon2 = coord_obs["Northeast Bound lonlat"][0]
    #lat2 = coord_obs["Northeast Bound lonlat"][1]
    return G.inv(lon1, lat1, lon2, lat2)[2]

def get_county_geodetic_distances_ur_centered(coord_obs,):
    lon1 = coord_obs["Northeast Bound lonlat"][0]
    lat1 = coord_obs["Northeast Bound lonlat"][1]
    lon2 = coord_obs["Longitude"]
    lat2 = coord_obs["Latitude"]
    return G.inv(lon1, lat1, lon2, lat2)[2]

def get_county_center(coord_obs,):
    lon1 = coord_obs["Southwest Bound lonlat"][0]
    lat1 = coord_obs["Southwest Bound lonlat"][1]
    lon2 = coord_obs["Northeast Bound lonlat"][0]
    lat2 = coord_obs["Northeast Bound lonlat"][1]
    x1,y1=lonlat_to_xy(lon1,lat1)
    x2,y2=lonlat_to_xy(lon2,lat2)
    x = (x1 + x2) / 2
    y = (y1 + y2) / 2
    lon, lat = xy_to_lonlat(x,y)
    return lon, lat

def get_county_geodetic_distances_ll_true_centered(coord_obs,):
    lon1 = coord_obs["Southwest Bound lonlat"][0]
    lat1 = coord_obs["Southwest Bound lonlat"][1]
    lon2 = coord_obs["true center"][0]
    lat2 = coord_obs["true center"][1]
    return G.inv(lon1, lat1, lon2, lat2)[2]

def get_county_geodetic_distances_ur_true_centered(coord_obs,):
    lon1 = coord_obs["Northeast Bound lonlat"][0]
    lat1 = coord_obs["Northeast Bound lonlat"][1]
    lon2 = coord_obs["true center"][0]
    lat2 = coord_obs["true center"][1]
    return G.inv(lon1, lat1, lon2, lat2)[2]

# get_county_distances, once again, results in projection distortions.
#coordinate_dataframe["meter_span"] = coordinate_dataframe.apply(get_county_distances,axis=1)
coordinate_dataframe["meter_span G"] = coordinate_dataframe.apply(get_county_geodetic_distances,axis=1)
coordinate_dataframe["meter_span G ll"] = coordinate_dataframe.apply(get_county_geodetic_distances_ll_centered,axis=1)
coordinate_dataframe["meter_span G ur"] = coordinate_dataframe.apply(get_county_geodetic_distances_ur_centered,axis=1)
coordinate_dataframe["true center"] = coordinate_dataframe.apply(get_county_center,axis=1)
coordinate_dataframe["meter_span G true ll"] = coordinate_dataframe.apply(get_county_geodetic_distances_ll_true_centered,axis=1)
coordinate_dataframe["meter_span G true ur"] = coordinate_dataframe.apply(get_county_geodetic_distances_ur_true_centered,axis=1)
coordinate_dataframe['county_radius'] = coordinate_dataframe[["meter_span G true ll","meter_span G true ur"]].max(axis=1)
#coordinate_dataframe["meter_span G"].apply(lambda dist: math.sqrt(dist[0]*dist[0] + dist[1]*dist[1]))

In [47]:
coordinate_dataframe[["meter_span G","meter_span G ll", "meter_span G ur","meter_span G true ll","meter_span G true ur","county_radius"]]

,meter_span G,meter_span G ll,meter_span G ur,meter_span G true ll,meter_span G true ur,county_radius
0,65354.544306,49787.767884,16543.318354,32631.350280,32723.578039,32723.578039
1,144988.408255,56285.977339,88703.487505,72088.270945,72903.258338,72903.258338
2,68727.167041,29602.439305,40704.123884,34316.694679,34410.906219,34410.906219
3,82058.574736,51037.686630,32737.684026,40960.733272,41098.528186,41098.528186
4,73203.215578,39378.094569,33860.417243,36551.288225,36652.445970,36652.445970
...,...,...,...,...,...,...
2863,58467.304180,28526.722507,30068.080131,29182.769167,29284.670788,29284.670788
2864,38931.970701,23152.953917,16021.873361,19450.045526,19481.976121,19481.976121
2865,101700.062886,43479.330208,58622.053825,50681.663084,51019.074061,51019.074061
2866,61739.543297,32232.412666,29929.947793,30820.000889,30919.720626,30919.720626


In [48]:
(coordinate_dataframe["county_radius"]  / 100000).apply(math.ceil).value_counts()

1     2801
2       57
3        4
4        3
5        2
14       1
Name: county_radius, dtype: int64

In [49]:
#coordinate_dataframe["Southwest Bound"].apply(lambda strlatlon: parse_string_latlon(strlatlon))#.apply(reverse_geocode.search)
#coordinates = (-37.81, 144.96), (31.76, 35.21)
#coordinate_dataframe["Southwest Bound"]
coordinate_dataframe['current_coordinates'] = coordinate_dataframe.apply(lambda x: tuple([x.Latitude,x.Longitude]),axis=1)
coordinate_dataframe['reverse_search_city'] = reverse_geocode.search(tuple(coordinate_dataframe['current_coordinates'].values))
coordinate_dataframe[["Area",'reverse_search_city']]
#reverse_geocode.search(tuple([(-37.81, 144.96),(-37.81, 144.96)]))

,Area,reverse_search_city
0,"Autauga, Alabama","{'country_code': 'US', 'city': 'Pine Level', '..."
1,"Baldwin, Alabama","{'country_code': 'US', 'city': 'Loxley', 'coun..."
2,"Bibb, Alabama","{'country_code': 'US', 'city': 'Centreville', ..."
3,"Blount, Alabama","{'country_code': 'US', 'city': 'Oneonta', 'cou..."
4,"Calhoun, Alabama","{'country_code': 'US', 'city': 'Weaver', 'coun..."
...,...,...
2863,"Gilmer, West Virginia","{'country_code': 'US', 'city': 'Newport', 'cou..."
2864,"Pleasants, West Virginia","{'country_code': 'US', 'city': 'Newport', 'cou..."
2865,"Pocahontas, West Virginia","{'country_code': 'US', 'city': 'Belmont Estate..."
2866,"Ritchie, West Virginia","{'country_code': 'US', 'city': 'Newport', 'cou..."


In [50]:
coordinate_dataframe["Longitude"] = coordinate_dataframe["true center"].apply(lambda coords: coords[0])
coordinate_dataframe["Latitude"] = coordinate_dataframe["true center"].apply(lambda coords: coords[1])

In [51]:
coordinate_dataframe=coordinate_dataframe.drop(["meter_span G","meter_span G ll",
                                                "meter_span G ur","meter_span G true ll",
                                                "meter_span G true ur","Southwest Bound lonlat",
                                               "Southwest Bound lonlat",
                                                "Northeast Bound lonlat",
                                               "Northeast Bound xy",
                                               "Southwest Bound xy",
                                               "current_coordinates",
                                                "Google County Name",
                                                "true center",'reverse_search_city'],axis=1)

In [52]:
coordinate_dataframe

,Area,Latitude,Longitude,Northeast Bound,Southwest Bound,county_radius
0,"Autauga, Alabama",32.508613,-86.667940,"(32.7082131, -86.411172)","(32.307574, -86.9212369)",32723.578039
1,"Baldwin, Alabama",30.731854,-87.710318,"(31.3190171, -87.366591)","(30.1465611, -88.0373089)",72903.258338
2,"Bibb, Alabama",33.040058,-87.150897,"(33.246926, -86.8760228)","(32.83152210000001, -87.421989)",34410.906219
3,"Blount, Alabama",34.013983,-86.636157,"(34.2604829, -86.3034978)","(33.765171, -86.963358)",41098.528186
4,"Calhoun, Alabama",33.763268,-85.839923,"(33.96846, -85.52943210000001)","(33.5559299, -86.146225)",36652.445970
...,...,...,...,...,...,...
2863,"Gilmer, West Virginia",38.913909,-80.832809,"(39.1094009, -80.60579899999999)","(38.717898, -81.05671389999999)",29284.670788
2864,"Pleasants, West Virginia",39.372666,-81.190293,"(39.4764319, -81.00783589999999)","(39.2683278, -81.371478)",19481.976121
2865,"Pocahontas, West Virginia",38.388464,-79.995256,"(38.739811, -79.61787509999999)","(38.03611, -80.3632959)",51019.074061
2866,"Ritchie, West Virginia",39.198974,-81.070768,"(39.38911909999999, -80.80888399999999)","(39.007884, -81.329228)",30919.720626


In [53]:
coordinate_dataframe.to_csv("Final County Coordinate Data.csv",index=False)